
# Kalman Filter Tutorial: From EKF to Observability-Aware Partial Updates

This tutorial walks through the progression from the Extended Kalman Filter (EKF)
to the Partial-Update Schmidt Kalman Filter (PSKF), and finally to the
Observability-informed Partial-Update Kalman Filter (OPSKF). We explore:

- The standard EKF for nonlinear state estimation
- Why partial updates are needed
- How observability affects estimation



## System: Nonlinear Van der Pol Oscillator

We consider the Van der Pol oscillator as our example system:

\[
\begin{aligned}
\dot{x}_1 &= x_2 \\
\dot{x}_2 &= \mu(1 - x_1^2)x_2 - x_1
\end{aligned}
\]

We discretize it for filtering. The measurement is only the first state:

\[
y_k = x_{1,k} + v_k
\]

where \( v_k \sim \mathcal{N}(0, R) \).


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Van der Pol dynamics
mu = 1.0
def f(x, u, t):
    return np.array([x[1], mu * (1 - x[0]**2) * x[1] - x[0]])

def h(x, u, t):
    return np.array([x[0]])


In [ ]:

# Discretize with forward Euler
def f_disc(x, u, t, dt=0.01):
    return x + f(x, u, t) * dt

def h_disc(x, u, t):
    return h(x, u, t)

# Simulate the system
T = 10
dt = 0.01
ts = np.arange(0, T, dt)
x0 = np.array([2.0, 0.0])
xs = [x0]
for t in ts[:-1]:
    xs.append(f_disc(xs[-1], None, t, dt))
xs = np.array(xs)
ys = xs[:, [0]] + np.random.normal(0, 0.1, size=(len(xs), 1))  # noisy measurement



## Extended Kalman Filter (EKF)

We implement the EKF using linearizations of \( f \) and \( h \) around the estimate.


In [ ]:

def jac_f(x, u, t):
    return np.array([
        [0, 1],
        [-2 * mu * x[0] * x[1] - 1, mu * (1 - x[0]**2)]
    ])

def jac_h(x, u, t):
    return np.array([[1, 0]])

Q = 0.01 * np.eye(2)
R = 0.1 * np.eye(1)
x_est = np.zeros((len(ts), 2))
P_est = np.zeros((len(ts), 2, 2))
x_hat = np.array([0.5, -1.0])
P = np.eye(2)

for k in range(len(ts)):
    # Predict
    F = jac_f(x_hat, None, ts[k])
    x_hat = f_disc(x_hat, None, ts[k], dt)
    P = F @ P @ F.T + Q

    # Update
    H = jac_h(x_hat, None, ts[k])
    y = ys[k]
    S = H @ P @ H.T + R
    K = P @ H.T @ np.linalg.inv(S)
    x_hat = x_hat + K @ (y - h_disc(x_hat, None, ts[k]))
    P = (np.eye(2) - K @ H) @ P

    x_est[k] = x_hat
    P_est[k] = P


In [ ]:

plt.figure(figsize=(10, 4))
plt.plot(ts, xs[:, 0], label="True $x_1$")
plt.plot(ts, ys, label="Measurement $y$")
plt.plot(ts, x_est[:, 0], label="EKF Estimate $\hat{x}_1$")
plt.xlabel("Time")
plt.legend()
plt.title("EKF Estimation")
plt.grid(True)
plt.tight_layout()
plt.show()



## Partial-Update Schmidt Kalman Filter (PSKF)

The PSKF allows only a subset of the state vector to be updated from the measurement.
Useful when some components are unobservable or when one wants to decouple estimation.

Let \( x = \begin{bmatrix} x_a \\ x_b \end{bmatrix} \) with:

- \( x_a \): directly observable states
- \( x_b \): indirectly inferred or nuisance states

Then only update \( x_a \), leaving \( x_b \) untouched.



## Observability-Aware Partial-Update Kalman Filter (OPSKF)

When observability is weak in certain directions, updates can degrade estimates.

OPSKF detects unobservable directions via the observability Gramian or the Jacobian nullspace,
and suppresses updates in those directions.

We define the linearized observability matrix:

\[
\mathcal{O}_k = \begin{bmatrix} H_k \\ H_k F_k \\ H_k F_k^2 \\ \cdots \end{bmatrix}
\]

Unobservable directions lie in the null space of \( \mathcal{O}_k \). In OPSKF,
we compute an orthogonal projector \( P_\mathrm{obs} \) and restrict the Kalman gain:

\[
K_k^\mathrm{OPSKF} = P_\mathrm{obs} K_k
\]

This preserves structure and robustness.
